In [1]:
#importing necessary modules

import pandas as pd
import tensorflow as tf

2022-07-24 16:20:35.960543: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Loading the datasets

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
# Checking Null values

train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# Drop unnecessary data

X = train.drop(["Name", "Ticket", "Cabin", "Embarked"], axis=1).dropna()
print(X.shape)
X.head()

(714, 8)


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


In [6]:
# Convert Categorical to Numerical

X["Sex"].replace("male", 1, inplace = True)
X["Sex"].replace("female", 0, inplace = True)
X.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,22.0,1,0,7.2500
1,2,1,1,0,38.0,1,0,71.2833
2,3,1,3,0,26.0,0,0,7.9250
3,4,1,1,0,35.0,1,0,53.1000
4,5,0,3,1,35.0,0,0,8.0500


In [7]:
## Create result dataset and drop result

y = X["Survived"]
X.drop(["Survived"], axis=1 , inplace=True)
y.head(), X.head()

(0    0
 1    1
 2    1
 3    1
 4    0
 Name: Survived, dtype: int64,
    PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare
 0            1       3    1  22.0      1      0   7.2500
 1            2       1    0  38.0      1      0  71.2833
 2            3       3    0  26.0      0      0   7.9250
 3            4       1    0  35.0      1      0  53.1000
 4            5       3    1  35.0      0      0   8.0500)

In [8]:
# Creating The Model

model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation=tf.keras.activations.linear),
    tf.keras.layers.Dense(4, activation=tf.keras.activations.linear),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
    ], name="Titanic-Model")

2022-07-24 16:20:58.419141: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-24 16:20:58.570932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-24 16:20:58.757133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 16:20:58.757455: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 5 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43GiB/s
2022-07-24 16:20:58.757555: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-24 16:20:59.369013: I tensorflow/stream_executor/platform/def

In [20]:
# Compile the model

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"])

In [21]:
# Train the Model

model.fit(X, y, epochs=200)

Epoch 1/200
23/23 [==============================] - 0s 1ms/step - loss: 29.0949 - accuracy: 0.5957
Epoch 2/200
23/23 [==============================] - 0s 1ms/step - loss: 22.8494 - accuracy: 0.4715
Epoch 3/200
23/23 [==============================] - 0s 1ms/step - loss: 18.7492 - accuracy: 0.4826
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 16.2610 - accuracy: 0.5047
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 13.5797 - accuracy: 0.4814
Epoch 6/200
23/23 [==============================] - 0s 2ms/step - loss: 12.3882 - accuracy: 0.4451
Epoch 7/200
23/23 [==============================] - 0s 2ms/step - loss: 9.0123 - accuracy: 0.5059
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 6.6594 - accuracy: 0.5057
Epoch 9/200
23/23 [==============================] - 0s 1ms/step - loss: 5.3607 - accuracy: 0.4805
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9499 - accuracy: 0.5243
Epo

In [22]:
model.evaluate(X,y)

23/23 [==============================] - 0s 1ms/step - loss: 0.4810 - accuracy: 0.7801


[0.48103851079940796, 0.7801120281219482]

In [23]:
# Function to load test data

def loadtestmodel(f):
    testing = pd.read_csv(f)
    testing.drop(["Name", "Ticket", "Cabin", "Embarked"], axis=1, inplace=True)
    testing["Sex"].replace("male", 1, inplace = True)
    testing["Sex"].replace("female", 0, inplace = True)
    return  testing


In [24]:
t = loadtestmodel("test.csv")

In [34]:
t.head(), t.shape

(   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare
 0          892       3    1  34.5      0      0   7.8292
 1          893       3    0  47.0      1      0   7.0000
 2          894       2    1  62.0      0      0   9.6875
 3          895       3    1  27.0      0      0   8.6625
 4          896       3    0  22.0      1      1  12.2875,
 (418, 7))

In [27]:
res = model.predict(t)
res

array([[0.3194411 ],
       [0.771604  ],
       [0.3420602 ],
       [0.35379148],
       [0.8365299 ],
       [0.41385707],
       [0.865424  ],
       [0.4439988 ],
       [0.8898238 ],
       [0.27433968],
       [       nan],
       [0.59822613],
       [0.97045046],
       [0.30274498],
       [0.94659925],
       [0.92023104],
       [0.47572482],
       [0.38325587],
       [0.8366107 ],
       [0.8314504 ],
       [0.54896736],
       [0.4065036 ],
       [       nan],
       [0.74349964],
       [0.98419327],
       [0.22050871],
       [0.9728406 ],
       [0.3806174 ],
       [0.6358614 ],
       [       nan],
       [0.36391142],
       [0.41862088],
       [0.80717856],
       [       nan],
       [0.66413635],
       [0.40196988],
       [       nan],
       [0.8902505 ],
       [0.3781223 ],
       [       nan],
       [0.30273852],
       [       nan],
       [0.30854383],
       [0.9290794 ],
       [0.94771194],
       [0.3780541 ],
       [0.6239603 ],
       [     

In [48]:
ans = []
for i in res:
    if i > 0.5:
        ans.append(1)
    else:
        ans.append(0)

In [58]:
test = pd.read_csv("test.csv")

In [59]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [60]:
test["Survived"] = ans

In [69]:
result=pd.DataFrame()

In [70]:
result["PassengerId"] = test["PassengerId"]
result["Survived"] = test["Survived"]

In [71]:
result.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [77]:
result = result.set_index("PassengerId")

In [78]:
result.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [80]:
result.to_csv("result.csv")